In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_40.csv')
pred2 = dd.read_csv('test_pred_46.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 33.1 s, sys: 3.05 s, total: 36.1 s
Wall time: 7.29 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 180 ms, sys: 297 ms, total: 476 ms
Wall time: 475 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 281 µs, sys: 35 µs, total: 316 µs
Wall time: 321 µs


In [7]:
coef_lgb = 0.85
coef_mlp = 0.15

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 267 ms, sys: 614 ms, total: 881 ms
Wall time: 877 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000183,0.362796,0.000366,0.193057,0.109242,0.000418,0.130937,0.000225,0.001534,0.015929,0.001995,0.182468,0.000267,0.000584,0.244229
3492886,130787971,0.000197,0.015786,0.000155,0.052439,0.159661,0.000241,0.020135,0.390422,0.000373,0.038864,0.000714,0.314093,0.000213,0.006706,0.234537
3492887,130787974,0.000678,0.022553,0.000134,0.857788,0.039060,0.000322,0.061305,0.000040,0.000148,0.001268,0.000381,0.013963,0.000202,0.002157,0.096807
3492888,130788053,0.000060,0.719354,0.000077,0.153495,0.003050,0.000108,0.010009,0.082802,0.000414,0.000291,0.000168,0.029565,0.000097,0.000510,0.163976
3492889,130788054,0.000208,0.098872,0.000305,0.434963,0.168438,0.000506,0.058640,0.000089,0.000605,0.021963,0.001196,0.212753,0.000674,0.000786,0.237050


In [13]:
%%time
test_ensemble.to_csv('test_pred_49.csv',index=False)

CPU times: user 1min 31s, sys: 1.27 s, total: 1min 33s
Wall time: 1min 33s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [ ]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_49.csv -m "Ensemble_LGB_MLP_085_015"